# EPyR Tools - Baseline Correction Tutorial

This notebook demonstrates advanced baseline correction techniques for EPR spectra using EPyR Tools.

## What You'll Learn

- Understanding baseline issues in EPR spectra
- Polynomial baseline correction (constant, linear, quadratic)
- Advanced correction with signal exclusion regions
- Comparing different correction methods
- Best practices for baseline correction

## Background

Baseline drift is a common issue in EPR spectroscopy caused by:
- Temperature changes during measurement
- Instrumental drift
- Sample heating effects
- Microwave frequency instability

Proper baseline correction is essential for:
- Accurate peak integration
- Reliable quantitative analysis
- Spectral comparison and subtraction

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add EPyR Tools to path
epyr_root = Path().resolve().parent.parent
if str(epyr_root) not in sys.path:
    sys.path.insert(0, str(epyr_root))

import epyr.eprload as eprload
from epyr.baseline import baseline_polynomial, baseline_exponential
from epyr.baseline._utils import find_baseline_regions

print("EPyR Tools baseline correction module loaded successfully!")

## 1. Loading Sample Data

Let's start by loading EPR data that shows baseline issues.

In [ ]:
# Try to load real EPR data first
data_dir = Path("../data")
sample_file = None

# Look for sample files
for file_path in (data_dir / "BES3T").glob("*.dsc"):
    sample_file = file_path
    break

if not sample_file:
    for file_path in (data_dir / "ESP").glob("*.par"):
        sample_file = file_path
        break

if sample_file:
    print(f"Loading real EPR data: {sample_file.name}")
    x, y, params, filepath = eprload.eprload(str(sample_file), plot_if_possible=False)
    
    if x is None or y is None:
        print("Failed to load real data. Using synthetic data instead.")
        sample_file = None

# Create synthetic data with realistic baseline issues
if sample_file is None:
    print("Creating synthetic EPR data with baseline drift...")
    
    # Generate field axis
    x = np.linspace(3200, 3400, 1000)
    
    # Create EPR signal (nitroxide radical with hyperfine splitting)
    centers = [3320, 3350, 3380]  # A_N = 30 G
    signal = np.zeros_like(x)
    
    for i, center in enumerate(centers):
        # Slightly different intensities for realism
        intensity = [80, 100, 75][i]  
        width = 8  # Gaussian linewidth
        signal += intensity * np.exp(-((x - center)**2) / (2 * width**2))
    
    # Add realistic baseline issues
    baseline_drift = (
        0.002 * (x - 3300)**2 +      # Quadratic drift (common)
        -0.15 * (x - 3300) +         # Linear drift  
        20 +                         # Constant offset
        5 * np.sin((x - 3200) * 0.02) # Sinusoidal modulation (rare but possible)
    )
    
    # Add noise
    noise = np.random.normal(0, 1.5, len(x))
    
    # Combine signal, baseline, and noise
    y = signal + baseline_drift + noise
    
    # Store true values for comparison
    y_true_signal = signal
    y_true_baseline = baseline_drift
    
    print(f"Created synthetic spectrum with {len(x)} points")
    print(f"Signal peak-to-peak: {np.ptp(signal):.1f}")
    print(f"Baseline drift: {np.ptp(baseline_drift):.1f}")

In [ ]:
# Visualize the original data
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(x, y, 'b-', linewidth=1.5, label='Raw spectrum (with baseline)')

# If synthetic, show the true components
if sample_file is None:
    plt.plot(x, y_true_signal, 'g:', linewidth=2, label='True EPR signal')
    plt.plot(x, y_true_baseline, 'r--', linewidth=1.5, alpha=0.7, label='True baseline')

plt.xlabel('Magnetic Field (G)')
plt.ylabel('EPR Signal (a.u.)')
plt.title('Original EPR Spectrum - Baseline Issues Visible')
plt.legend()
plt.grid(True, alpha=0.3)

# Show first derivative for better peak visibility
plt.subplot(2, 1, 2)
dy_dx = np.gradient(y, x)
plt.plot(x, dy_dx, 'b-', linewidth=1.5, label='First derivative')
plt.xlabel('Magnetic Field (G)')
plt.ylabel('dχ"/dB (a.u.)')
plt.title('First Derivative - Shows Peak Positions Despite Baseline')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nOriginal spectrum statistics:")
print(f"Mean: {np.mean(y):.2f}")
print(f"Std: {np.std(y):.2f}")
print(f"Range: {np.min(y):.2f} to {np.max(y):.2f}")

## 2. Polynomial Baseline Correction

The most common approach is fitting a polynomial to the baseline regions.

In [ ]:
# Apply different polynomial orders
corrections = {}

print("Applying polynomial baseline corrections...")

# Constant offset (0th order)
y_const, baseline_const = baseline_polynomial(y, x_data=x, poly_order=0)
corrections['Constant (0th order)'] = (y_const, baseline_const)
print(f"✓ Constant correction: offset = {baseline_const[0]:.2f}")

# Linear (1st order)
y_linear, baseline_linear = baseline_polynomial(y, x_data=x, poly_order=1)
corrections['Linear (1st order)'] = (y_linear, baseline_linear)
slope = (baseline_linear[-1] - baseline_linear[0]) / (x[-1] - x[0])
print(f"✓ Linear correction: slope = {slope:.4f} a.u./G")

# Quadratic (2nd order)
y_quad, baseline_quad = baseline_polynomial(y, x_data=x, poly_order=2)
corrections['Quadratic (2nd order)'] = (y_quad, baseline_quad)
print(f"✓ Quadratic correction applied")

# Cubic (3rd order) - be careful with higher orders!
y_cubic, baseline_cubic = baseline_polynomial(y, x_data=x, poly_order=3)
corrections['Cubic (3rd order)'] = (y_cubic, baseline_cubic)
print(f"✓ Cubic correction applied")

print(f"\nApplied {len(corrections)} different correction methods.")

In [ ]:
# Create comprehensive comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

methods = list(corrections.keys())

for i, method in enumerate(methods):
    ax = axes[i]
    y_corr, baseline_fit = corrections[method]
    
    # Plot original and corrected spectra
    ax.plot(x, y, 'lightgray', linewidth=1, alpha=0.7, label='Original')
    ax.plot(x, baseline_fit, 'r--', linewidth=1.5, alpha=0.8, label='Fitted baseline')
    ax.plot(x, y_corr, 'b-', linewidth=1.5, label='Corrected')
    
    # If synthetic data, show true baseline for comparison
    if sample_file is None:
        ax.plot(x, y_true_baseline, 'k:', linewidth=1, alpha=0.6, label='True baseline')
    
    ax.set_xlabel('Magnetic Field (G)')
    ax.set_ylabel('EPR Signal (a.u.)')
    ax.set_title(f'{method} Correction')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=9)
    
    # Calculate and display RMS of residuals
    residuals = y - baseline_fit
    rms = np.sqrt(np.mean(residuals**2))
    ax.text(0.02, 0.98, f'RMS: {rms:.2f}', transform=ax.transAxes,
            verticalalignment='top', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('Polynomial Baseline Correction Comparison', fontsize=16)
plt.tight_layout()
plt.show()

# Calculate quality metrics
print("\nCorrection Quality Metrics:")
print("=" * 40)
print(f"{'Method':<20} {'RMS':<8} {'Signal Std':<12} {'Mean':<10}")
print("-" * 50)

for method, (y_corr, baseline_fit) in corrections.items():
    residuals = y - baseline_fit
    rms = np.sqrt(np.mean(residuals**2))
    signal_std = np.std(y_corr)
    signal_mean = np.mean(y_corr)
    print(f"{method:<20} {rms:<8.2f} {signal_std:<12.2f} {signal_mean:<10.2f}")

## 3. Advanced Correction with Signal Exclusion

When EPR signals are strong, they can bias the baseline fit. We can exclude signal regions from the baseline calculation.

In [ ]:
# Identify signal regions automatically
print("Identifying signal regions for exclusion...")

# Method 1: Based on signal strength
signal_threshold = np.std(y) * 2  # Points above 2 sigma
baseline_mean = np.mean(y)

# Find regions where signal deviates significantly from mean
signal_mask = np.abs(y - baseline_mean) > signal_threshold
signal_indices = np.where(signal_mask)[0]

print(f"Found {len(signal_indices)} points above threshold ({signal_threshold:.2f})")

# Group consecutive points into regions
if len(signal_indices) > 0:
    # Find breaks in consecutive indices
    breaks = np.where(np.diff(signal_indices) > 10)[0] + 1  # Gap > 10 points
    region_starts = np.concatenate(([0], breaks))
    region_ends = np.concatenate((breaks, [len(signal_indices)]))
    
    exclude_regions = []
    field_width = (x.max() - x.min()) / len(x) * 20  # Buffer around signals
    
    for start, end in zip(region_starts, region_ends):
        idx_start = signal_indices[start]
        idx_end = signal_indices[end-1]
        
        field_start = max(x[idx_start] - field_width, x.min())
        field_end = min(x[idx_end] + field_width, x.max())
        
        exclude_regions.append((field_start, field_end))
    
    print(f"Created {len(exclude_regions)} exclusion regions:")
    for i, (start, end) in enumerate(exclude_regions):
        print(f"  Region {i+1}: {start:.1f} - {end:.1f} G ({end-start:.1f} G width)")
else:
    exclude_regions = []
    print("No significant signal regions found for exclusion.")

In [ ]:
# Apply baseline correction with exclusion regions
if exclude_regions:
    print("\nApplying corrections with signal exclusion...")
    
    # Compare with and without exclusion
    corrections_excl = {}
    
    # Linear with exclusion
    y_lin_excl, baseline_lin_excl = baseline_polynomial(
        y, x_data=x, poly_order=1, exclude_regions=exclude_regions
    )
    corrections_excl['Linear + Exclusion'] = (y_lin_excl, baseline_lin_excl)
    
    # Quadratic with exclusion
    y_quad_excl, baseline_quad_excl = baseline_polynomial(
        y, x_data=x, poly_order=2, exclude_regions=exclude_regions
    )
    corrections_excl['Quadratic + Exclusion'] = (y_quad_excl, baseline_quad_excl)
    
    # Create comparison plot
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # Original spectrum with exclusion regions highlighted
    ax = axes[0]
    ax.plot(x, y, 'b-', linewidth=1.5, label='Original spectrum')
    ax.plot(x, baseline_quad_excl, 'r--', linewidth=2, label='Baseline (with exclusions)')
    
    # Highlight excluded regions
    for i, (start, end) in enumerate(exclude_regions):
        label = 'Excluded regions' if i == 0 else ""
        ax.axvspan(start, end, alpha=0.3, color='red', label=label)
    
    ax.set_xlabel('Magnetic Field (G)')
    ax.set_ylabel('EPR Signal (a.u.)')
    ax.set_title('Baseline Fitting with Signal Exclusion')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Compare corrections with and without exclusion
    ax = axes[1]
    ax.plot(x, y_quad, 'g-', linewidth=1.5, alpha=0.7, label='Quadratic (no exclusion)')
    ax.plot(x, y_quad_excl, 'b-', linewidth=1.5, label='Quadratic + exclusion')
    
    if sample_file is None:  # Show true signal for synthetic data
        ax.plot(x, y_true_signal, 'k:', linewidth=2, alpha=0.8, label='True signal')
    
    ax.set_xlabel('Magnetic Field (G)')
    ax.set_ylabel('EPR Signal (a.u.)')
    ax.set_title('Corrected Spectra Comparison')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Show the difference
    ax = axes[2]
    difference = y_quad_excl - y_quad
    ax.plot(x, difference, 'm-', linewidth=1.5, label='Difference (exclusion - standard)')
    ax.axhline(y=0, color='k', linestyle=':', alpha=0.5)
    ax.set_xlabel('Magnetic Field (G)')
    ax.set_ylabel('Signal Difference (a.u.)')
    ax.set_title('Effect of Signal Exclusion')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Quality comparison
    print("\nComparison of exclusion methods:")
    rms_standard = np.sqrt(np.mean((y - baseline_quad)**2))
    rms_exclusion = np.sqrt(np.mean((y - baseline_quad_excl)**2))
    
    print(f"Standard quadratic RMS: {rms_standard:.3f}")
    print(f"With exclusion RMS: {rms_exclusion:.3f}")
    print(f"Improvement: {((rms_standard - rms_exclusion)/rms_standard)*100:.1f}%")
    
    if sample_file is None:
        # Compare with true signal for synthetic data
        mse_standard = np.mean((y_quad - y_true_signal)**2)
        mse_exclusion = np.mean((y_quad_excl - y_true_signal)**2)
        print(f"\nMSE vs true signal:")
        print(f"Standard: {mse_standard:.3f}")
        print(f"Exclusion: {mse_exclusion:.3f}")
        print(f"True signal recovery improvement: {((mse_standard - mse_exclusion)/mse_standard)*100:.1f}%")

else:
    print("Skipping exclusion demonstration (no clear signal regions detected).")

## 4. Alternative Baseline Correction Methods

EPyR Tools also supports other baseline correction approaches.

In [ ]:
# Try exponential baseline correction (useful for some EPR artifacts)
print("Testing exponential baseline correction...")

try:
    y_exp, baseline_exp = baseline_exponential(y, x_data=x)
    
    # Compare exponential vs polynomial
    plt.figure(figsize=(14, 10))
    
    # Original spectrum
    plt.subplot(3, 1, 1)
    plt.plot(x, y, 'b-', linewidth=1.5, label='Original')
    plt.plot(x, baseline_quad, 'r--', linewidth=1.5, alpha=0.8, label='Polynomial baseline')
    plt.plot(x, baseline_exp, 'g--', linewidth=1.5, alpha=0.8, label='Exponential baseline')
    
    if sample_file is None:
        plt.plot(x, y_true_baseline, 'k:', linewidth=2, alpha=0.6, label='True baseline')
    
    plt.xlabel('Magnetic Field (G)')
    plt.ylabel('EPR Signal (a.u.)')
    plt.title('Baseline Fitting Methods Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Corrected spectra
    plt.subplot(3, 1, 2)
    plt.plot(x, y_quad, 'r-', linewidth=1.5, label='Polynomial corrected')
    plt.plot(x, y_exp, 'g-', linewidth=1.5, label='Exponential corrected')
    
    if sample_file is None:
        plt.plot(x, y_true_signal, 'k:', linewidth=2, alpha=0.8, label='True signal')
    
    plt.xlabel('Magnetic Field (G)')
    plt.ylabel('EPR Signal (a.u.)')
    plt.title('Corrected Spectra')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Difference between methods
    plt.subplot(3, 1, 3)
    diff_methods = y_exp - y_quad
    plt.plot(x, diff_methods, 'm-', linewidth=1.5, label='Exponential - Polynomial')
    plt.axhline(y=0, color='k', linestyle=':', alpha=0.5)
    plt.xlabel('Magnetic Field (G)')
    plt.ylabel('Signal Difference (a.u.)')
    plt.title('Difference Between Correction Methods')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Method comparison statistics
    rms_poly = np.sqrt(np.mean((y - baseline_quad)**2))
    rms_exp = np.sqrt(np.mean((y - baseline_exp)**2))
    
    print(f"\nMethod comparison:")
    print(f"Polynomial RMS: {rms_poly:.3f}")
    print(f"Exponential RMS: {rms_exp:.3f}")
    
    if rms_exp < rms_poly:
        print(f"Exponential method is {((rms_poly - rms_exp)/rms_poly)*100:.1f}% better")
    else:
        print(f"Polynomial method is {((rms_exp - rms_poly)/rms_exp)*100:.1f}% better")
        
except Exception as e:
    print(f"Exponential correction failed: {e}")
    print("This is normal for some data types - polynomial methods are more robust.")

## 5. Best Practices and Guidelines

Here are some practical tips for baseline correction in EPR spectroscopy.

In [ ]:
# Demonstrate best practices with interactive examples
print("EPR Baseline Correction Best Practices")
print("=" * 40)

# 1. Choose appropriate polynomial order
print("\n1. Choosing Polynomial Order:")
orders = [0, 1, 2, 3, 4, 5]
rms_values = []

for order in orders:
    try:
        _, baseline_fit = baseline_polynomial(y, x_data=x, poly_order=order)
        rms = np.sqrt(np.mean((y - baseline_fit)**2))
        rms_values.append(rms)
        print(f"  Order {order}: RMS = {rms:.3f}")
    except:
        rms_values.append(np.inf)
        print(f"  Order {order}: Failed (overfitting)")

optimal_order = orders[np.argmin(rms_values)]
print(f"  → Optimal order for this data: {optimal_order}")

# Plot RMS vs order
plt.figure(figsize=(10, 6))
valid_orders = [o for o, r in zip(orders, rms_values) if r != np.inf]
valid_rms = [r for r in rms_values if r != np.inf]

plt.plot(valid_orders, valid_rms, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=optimal_order, color='r', linestyle='--', alpha=0.7, label=f'Optimal: Order {optimal_order}')
plt.xlabel('Polynomial Order')
plt.ylabel('RMS of Residuals')
plt.title('Baseline Correction Quality vs Polynomial Order')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("\n2. When to Use Exclusion Regions:")
signal_to_baseline = np.ptp(y_true_signal) / np.ptp(y_true_baseline) if sample_file is None else "Unknown"
if isinstance(signal_to_baseline, (int, float)):
    print(f"  Signal-to-baseline ratio: {signal_to_baseline:.2f}")
    if signal_to_baseline > 0.1:
        print("  → Recommendation: Use exclusion regions (strong signals can bias baseline)")
    else:
        print("  → Recommendation: Standard correction sufficient (weak signals)")
else:
    print("  → For real data: Use exclusion if EPR signals are > 10% of baseline drift")

print("\n3. Quality Assessment:")
y_final = y_quad_excl if 'y_quad_excl' in locals() else y_quad
baseline_final = baseline_quad_excl if 'baseline_quad_excl' in locals() else baseline_quad

# Check for over-correction signs
corrected_mean = np.mean(y_final)
corrected_std = np.std(y_final)
corrected_trend = np.polyfit(x, y_final, 1)[0]  # Remaining linear trend

print(f"  Corrected spectrum mean: {corrected_mean:.3f} (should be near zero)")
print(f"  Remaining linear trend: {corrected_trend:.6f} (should be near zero)")
print(f"  Signal-to-noise ratio: {np.ptp(y_final) / corrected_std:.1f}")

if abs(corrected_trend) < 0.001:
    print("  ✓ Good correction: minimal remaining trend")
else:
    print("  ⚠ Possible under-correction: linear trend remains")

if abs(corrected_mean) < corrected_std:
    print("  ✓ Good correction: mean near zero")
else:
    print("  ⚠ Possible over-correction: mean shifted from zero")

print("\n4. Method Selection Guidelines:")
print("  • Constant (order 0): Simple DC offset removal")
print("  • Linear (order 1): Most common, handles typical drift")
print("  • Quadratic (order 2): For curved baselines, temperature effects")
print("  • Higher orders: Use with caution, risk of over-correction")
print("  • Exclusion regions: When signals are strong relative to baseline")
print("  • Exponential: For specific instrumental artifacts (rare)")

In [ ]:
# Save the corrected spectrum for further analysis
output_dir = Path("../data/processed")
output_dir.mkdir(exist_ok=True)

# Use the best correction method
y_best = y_quad_excl if 'y_quad_excl' in locals() else y_quad
baseline_best = baseline_quad_excl if 'baseline_quad_excl' in locals() else baseline_quad
method_name = "quadratic_with_exclusion" if 'y_quad_excl' in locals() else "quadratic"

# Save as CSV for easy import into other software
output_file = output_dir / f"baseline_corrected_{method_name}.csv"

import pandas as pd

df = pd.DataFrame({
    'field_gauss': x,
    'original_intensity': y,
    'fitted_baseline': baseline_best,
    'corrected_intensity': y_best
})

# Add metadata as comments
with open(output_file, 'w') as f:
    f.write(f"# EPR Baseline Correction Results\n")
    f.write(f"# Method: {method_name}\n")
    f.write(f"# RMS residual: {np.sqrt(np.mean((y - baseline_best)**2)):.4f}\n")
    f.write(f"# Corrected mean: {np.mean(y_best):.4f}\n")
    f.write(f"# Generated by: EPyR Tools Baseline Correction Tutorial\n")
    f.write(f"#\n")

df.to_csv(output_file, mode='a', index=False)
print(f"\nCorrected spectrum saved: {output_file}")

# Final summary plot
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(x, y, 'lightgray', linewidth=1, alpha=0.8, label='Original')
plt.plot(x, baseline_best, 'r--', linewidth=2, alpha=0.8, label='Fitted baseline')
plt.plot(x, y_best, 'b-', linewidth=2, label=f'Corrected ({method_name})')

if sample_file is None:
    plt.plot(x, y_true_signal, 'g:', linewidth=2, alpha=0.8, label='True signal')

plt.xlabel('Magnetic Field (G)')
plt.ylabel('EPR Signal (a.u.)')
plt.title('Final Baseline Correction Result')
plt.legend()
plt.grid(True, alpha=0.3)

# First derivative of corrected spectrum
plt.subplot(2, 1, 2)
dy_corrected = np.gradient(y_best, x)
plt.plot(x, dy_corrected, 'b-', linewidth=1.5, label='Corrected derivative')
plt.xlabel('Magnetic Field (G)')
plt.ylabel('dχ"/dB (a.u.)')
plt.title('First Derivative of Corrected Spectrum')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Baseline correction tutorial completed successfully!")
print(f"\nNext steps:")
print(f"- Load {output_file} for quantitative analysis")
print(f"- Try the Advanced Analysis tutorial for peak fitting")
print(f"- Process multiple files with Batch Processing tutorial")

## Summary

This tutorial covered:

### Key Concepts
- **Baseline issues**: Common in EPR due to instrumental drift and temperature effects
- **Polynomial correction**: Most versatile approach with different orders for different drift patterns
- **Signal exclusion**: Important when EPR signals are strong relative to baseline
- **Method selection**: Based on data characteristics and correction quality

### Best Practices
1. **Start simple**: Try linear correction first
2. **Assess quality**: Check RMS, remaining trends, and mean values
3. **Use exclusion**: When signals > 10% of baseline drift
4. **Avoid over-correction**: Higher polynomial orders can distort signals
5. **Visual inspection**: Always plot results to verify correction quality

### When to Use Each Method
- **Constant (order 0)**: Simple DC offset, very stable measurements
- **Linear (order 1)**: Most EPR measurements, gradual drift
- **Quadratic (order 2)**: Temperature effects, longer measurements
- **With exclusion**: Strong signals, quantitative analysis
- **Exponential**: Specific instrumental artifacts (rare)

### Next Tutorials
- **03_Advanced_Analysis.ipynb**: Peak fitting, integration, g-factor calculations
- **04_Batch_Processing.ipynb**: Process multiple EPR files efficiently

The corrected spectrum is now ready for quantitative EPR analysis!